In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
#
# Extracts exceptions from log files.
#
import sys
import re
from collections import defaultdict

In [2]:
EXCEPTION_TAGS = ['Exception']
SKIP_TAGS = ['activeException']
CONTINUE_TAGS = [' at ', '\tat ', ' Caused by: ', '.java:']
END_TAGS = ['frames omitted']
ERR_TAGS = [' WARN ', ' ERROR ']
REGEX = 'Regex'
DESC = 'Description'
REF = 'Reference'
CAUSE = 'Cause'
IMPACT = 'Impact'
# Usually, all inner lines of a stack trace will be "at" or "Caused by" lines.
# With one exception: the line following a "nested exception is" line does not
# follow that convention. Due to that, this line is handled separately.
# CONT = re.compile("; nested exception is: *$")
exceptions = defaultdict(int)
errorLogs = defaultdict(int)
configDict = defaultdict(str)
signature = {}
RuleIDs = None

In [3]:
def registerKeyValue(key,value):
    configDict[key] += value

In [4]:
def registerException(exc):
    exceptions[exc] += 1

In [5]:
def registerErrors(err):
    errorLogs[err] += 1

In [6]:
def substituteX(regex, var, string):
    string = re.sub(regex,var,string)
    return str(string)

In [7]:
def removeUnwanted(fromString, string):
    string = string.split(fromString)
    return str( re.sub('\s+',' ',fromString) + string[1])

In [8]:
def containsList(varList, string):
    matchFlag = False
    for item in varList:
        if item in string:
            matchFlag = True
            break
    #   print ('%s matchFlag, %s item, %s string' %(matchFlag, item, string)) 
    return matchFlag, item

In [9]:
def addSignatureDetails(rule):
    rule += '.'
    details = "%s: %s,\n %s: %s,\n %s:%s,\n %s:%s" %(DESC, configDict.get(rule+DESC), REF, configDict.get(rule+REF), CAUSE, configDict.get(rule+CAUSE), IMPACT, configDict.get(rule+IMPACT))
    return str(details) 

In [10]:
def loadInputFile(fileName):
    global RuleIDs
    with open(fileName, "r") as fh:
        data = fh.read()
    pattern = re.compile(r'(\w+.\w+)[\s]*=[\s]*(.*)', re.MULTILINE)
    matches = [m.groups() for m in pattern.finditer(data)]
    for m in matches:
        registerKeyValue(m[0],m[1])
    data = None
    print(data)
    RuleIDs = configDict.get('RuleIDs').split(",")

In [11]:
#configDict

In [12]:
def processFile(fileName):
    with open(fileName, "r",encoding='utf-16') as fh:
        currentMatch = None
        addNextLine = False
        endCounter = 0
        for line in fh.readlines():
            skipMatch = [True for x in SKIP_TAGS if x in line]
            if skipMatch:
                continue
            errorMatch = [True for x in ERR_TAGS if x in line]
            if errorMatch:
                if addNextLine and currentMatch != None:
                    registerException(currentMatch)
                    endCounter = 0
                    currentMatch = None
                    addNextLine = False
                line = substituteX('[0-9]+', 'X', line)
                line = substituteX('\[(.*?)\]', 'X', line)
                registerErrors(line)
                continue
            exceptionFlag, matchedException = containsList(EXCEPTION_TAGS, line)
            continueFlag, matchedContinue = containsList(CONTINUE_TAGS, line)
            endFlag, matchedEnd = containsList(END_TAGS, line)
    #      print ('%s exceptionFlag, %s continueFlag, %s endFlag, %d endCounter, %s matchedEnd, %s addNextLine, %s line' %(exceptionFlag, continueFlag, endFlag, endCounter, matchedEnd, addNextLine, line))
            if exceptionFlag and addNextLine and continueFlag:
                currentMatch += removeUnwanted(matchedContinue,line)
            elif exceptionFlag and addNextLine:
                currentMatch += substituteX('[0-9]+', 'X', line)  
            elif exceptionFlag:
                currentMatch = substituteX('[0-9]+', 'X', line)
                addNextLine = True
                continue
            elif continueFlag and addNextLine:
                currentMatch += removeUnwanted(matchedContinue,line)
            elif endFlag and endCounter < 2 and addNextLine:
                currentMatch += removeUnwanted('...',line)
                endCounter += 1
            else:
                addNextLine, matchedEnd = containsList(['.java:'], line)
                if addNextLine:
                    currentMatch += substituteX('[0-9]+', 'X', line)
                else:
                    if currentMatch != None:
                        registerException(currentMatch)
                    endCounter = 0
                    currentMatch = None
                    addNextLine = False

In [13]:
def matchSignatures(logs):
    for log, n in logs.items():
        for ruleid in RuleIDs:
#         matched = re.findall(configDict.get(ruleid+".Regex"), exception, re.DOTALL|re.MULTILINE)
            matched = re.search(configDict.get(ruleid+'.'+REGEX), log, re.DOTALL|re.MULTILINE)
#         matched = re.compile(configDict.get(ruleid+".Regex"), re.MULTILINE).match(exception)
#         print "Trying to match rule:",ruleid,' with regex:',configDict.get(ruleid+".Regex"),' and string:', exception, 'outcome is:', matched
            if not matched:
                signature[log] = [n, 'NO SIGNATURE DEFINITION FOUND!!!' ]
                continue
            else:
                signature[log] = [n, addSignatureDetails(ruleid)]
                matched = None
                break

In [14]:
signature

{}

In [15]:
def createHTMLPage():
    content = "<style type='text/css'>.myTable { background-color:#eee;border-collapse:collapse; }.myTable th { background-color:#000;color:white; }.myTable td, .myTable th { padding:5px;border:1px solid #000; }</style>"
    content += "<html><table class='myTable'><tr><th>Count</th><th>Exception/Error</th><th>Description</th></tr>"
    print(sorted(signature, key=lambda item: (signature[item], item), reverse=True))
    for sign in sorted(signature, key=lambda item: (signature[item], item), reverse=True):
        print(sign)
        print("Check",signature[sign][1])
        content += "<tr><td>%s</td><td>%s</td><td>%s</td>" %(signature[sign][0],substituteX('\n','<br/>',sign),substituteX('\n','<br/>',signature[sign][1]))
    content += "</table></html>"
    htm = open("HTMLTable.html", mode='w')
    htm.write(content)
    #   print content
    htm.close()

In [16]:
loadInputFile(r"C:\sunil\python files\signatures.txt")

None


In [17]:
processFile(r"C:\CodeAnalytics\Log\POD2_Rest.log")

In [18]:
matchSignatures(exceptions)
matchSignatures(errorLogs)

In [19]:
createHTMLPage()

['X X-X-X X:X:X,X X WARN  c.c.p.e.client.bsf.BsfRequestClient - No endpoint configured for local profile for BSF \n', 'X X-X-X X:X:X,X X ERROR c.c.p.nf.cache.NrfServiceRepository - IOException while Subscription \n', '[pcf-rest-ep-XbXbX-crbmt] java.io.IOException: java.util.concurrent.ExecutionException: java.net.SocketTimeoutException: Connect Timeout \n at com.cisco.pcf.endpoint.client.Http2JettyRequest.execute(Http2JettyRequest.java:61) \n at com.google.api.client.http.HttpRequest.execute(HttpRequest.java:981) \n at com.cisco.pcf.endpoint.client.GenericRestClient.executePatchRequest(GenericRestClient.java:41) \n at com.cisco.pcf.nf.cache.NrfServiceRepository$NrfSubscription.run(NrfServiceRepository.java:125) \n at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511) \n at java.util.concurrent.FutureTask.runAndReset(FutureTask.java:308) \n at java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.access$301(ScheduledThreadPoolExecutor.java:180) \n at 

In [18]:
txt = '''[pcf-rest-ep-6c7ffbdc4c-d2ph6] (nfInstanceId=897d3e19-035d-49c1-9d04-8cb072626420, serviceName=nchf-spendinglimitcontrol, uris=[http://10.195.17.3:1090/nchf-spendinglimitcontrol/v1], priority=1001, capacity=150)'''
pattern = re.compile(r'(\w+.\w+)[\s]*=[\s]*(.*)', re.MULTILINE)
matches = [m.groups() for m in pattern.finditer(txt)]
print(matches)
for n in matches:
    print('match',n,end='\n----\n')
    registerKeyValue(n[0],n[1])

[('nfInstanceId', '897d3e19-035d-49c1-9d04-8cb072626420, serviceName=nchf-spendinglimitcontrol, uris=[http://10.195.17.3:1090/nchf-spendinglimitcontrol/v1], priority=1001, capacity=150)')]
match ('nfInstanceId', '897d3e19-035d-49c1-9d04-8cb072626420, serviceName=nchf-spendinglimitcontrol, uris=[http://10.195.17.3:1090/nchf-spendinglimitcontrol/v1], priority=1001, capacity=150)')
----


In [ ]:
,encoding='utf-16'